In [ ]:
#這是連接GCP的mysql的程式 其中參數的db不用代入
#本地密碼是Allenlee7@
#GCP的密碼是allenlee7
import pandas as pd
import pymysql #pymysql 模組，是用來操作 MySQL 資料庫的套件

# 設定資料庫連線資訊
host = 'localhost'  # 資料庫主機地址
# 如果是遠端主機，請使用遠端主機的 IP 地址或域名
# 如果是本機，則可以使用 'localhost' 或 '127.0.0.1'

port = 3306     # MySQL 預設的連接埠號是 3306
user = 'root'   #使用者名稱，例如 'root'（管理者帳號）

passwd = 'allenlee7'   # 資料庫密碼，請根據實際情況修改
db = 'test'           # 資料庫名稱，請根據實際情況修改
charset = 'utf8mb4'     # 字符集，通常使用 'utf8mb4' 以支援完整 Unicode（包括 emoji）

# 建立連線
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, charset=charset)
#使用 pymysql.connect(...) 來建立一個資料庫連線物件 conn
#此連線物件可用於：建立 cursor 游標查詢、執行 SQL 指令（查詢、更新、插入等）、提交或回滾交易（commit() / rollback()）

print('Successfully connected!')

Successfully connected!


In [6]:
#讀取資料，確定data在mysql中已建立
df = pd.read_sql(
    "select * from world",
    conn,
)

print(df)

          name continent     area  population             gdp
0  Afghanistan      Asia   652230    25500100   2034300000000
1      Albania    Europe    28748     2831741   1296000000000
2      Algeria    Africa  2381741    37100000   1886810000000
3      Andorra    Europe      468       78115    371200000000
4       Angola    Africa  1246700    20609294  10099000000000


C:\Users\Legato\AppData\Local\Temp\ipykernel_57460\511623586.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [7]:
#python寫法，並使用連接mysql的data，做進一步的篩選資料
def big_countries1(world: pd.DataFrame) -> pd.DataFrame:
    return(
        world.loc[
            (world["area"]>=3000000) | (world["population"]>=25000000) ,["name","population","area"]
        ]
    )
    
big_countries1(df)

,name,population,area
0,Afghanistan,25500100,652230
2,Algeria,37100000,2381741


In [8]:
#在python直接使用mysql的寫法：
cursor = conn.cursor()      
#conn 是之前使用 pymysql.connect() 建立的資料庫連線對象
#創建一個游標（cursor）對象，該對象用於在資料庫中執行 SQL 查詢並處理結果

sql = """
select name,population,area 
from world 
where area>=3000000 or population>=25000000;
"""
#定義多行 SQL 查詢語句，儲存在 sql 變數中

cursor.execute(sql)
#execute(sql) 是游標對象的方法，接受一個 SQL 語句作為參數，並將其發送到資料庫伺服器執行，執行後，查詢結果被緩存在游標中

cursor.scroll(0, mode='absolute')
#0 表示偏移量，mode='absolute' 表示將游標移動到結果集的第 0 行（第一行）
#確保游標從頭開始，特別是在多次執行或滾動後重置位置

world = pd.DataFrame(cursor.fetchall())
#fetchall() 是游標的方法，提取查詢結果集中的所有行，返回一個列表，每行是一個元組，包含查詢選取的欄位值
#然後將其轉為dataframe的形式
print(world)

             0         1        2
0  Afghanistan  25500100   652230
1      Algeria  37100000  2381741


In [9]:
#將游標與連結關閉
cursor.close()
conn.close()